In [2]:
import env
import acquire
import split_scale
import pandas as pd
import numpy as np

In [2]:
df_iris = acquire.wrangle_iris()

   measurement_id  sepal_length  sepal_width  petal_length  petal_width  \
0               1           5.1          3.5           1.4          0.2   
1               2           4.9          3.0           1.4          0.2   
2               3           4.7          3.2           1.3          0.2   

   species_id  species_id species_name  
0           1           1       setosa  
1           1           1       setosa  
2           1           1       setosa  
(150, 8)
Index(['measurement_id', 'sepal_length', 'sepal_width', 'petal_length',
       'petal_width', 'species_id', 'species_id', 'species_name'],
      dtype='object')
measurement_id      int64
sepal_length      float64
sepal_width       float64
petal_length      float64
petal_width       float64
species_id          int64
species_id          int64
species_name       object
dtype: object
       measurement_id  sepal_length  sepal_width  petal_length  petal_width  \
count      150.000000    150.000000   150.000000    150.000000  

In [3]:
df_excel, df_excel_sample = acquire.excel_reader()

Index(['customer_id', 'gender', 'is_senior_citizen', 'partner', 'dependents'], dtype='object')
customer_id     object
gender          object
partner         object
dependents      object
payment_type    object
churn           object
dtype: object
     is_senior_citizen  phone_service  internet_service  contract_type  \
min                0.0            0.0               0.0            0.0   
max                1.0            2.0               2.0            2.0   

     monthly_charges  total_charges  
min            18.25           18.8  
max           118.75         8684.8  


In [4]:
df_google, unique_categories = acquire.google_sheet()

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int

yay


In [7]:
print('y')

y


In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import split_scale
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import wrangle
import env
import seaborn as sns

url = env.get_db_url('iris_db')
#
def wrangle_iris():
    df = pd.read_sql("""

SELECT *
FROM measurements m
JOIN species s on s.species_id = m.species_id;
"""
,url)
    print(df.head(3))
    
    print(df.shape)
    
    print(df.columns)
    
    print(df.dtypes)
    
    print(df.describe())
    return df


def excel_reader():
    df_excel = pd.read_excel('Excel_Exercises.xlsx',sheet_name='Table1_CustDetails')
    df_excel_sample = pd.read_excel('Excel_Exercises.xlsx',sheet_name='Table1_CustDetails',nrows=100)
    print(df_excel.columns[0:5])
    print(df_excel.dtypes[df_excel.dtypes == object])
    print(df_excel.describe().loc[['min','max']])
    return df_excel, df_excel_sample


def google_sheet():
    google_sheet = "https://docs.google.com/spreadsheets/d/1Uhtml8KY19LILuZsrDtlsHHDC9wuDGUSe8LTEwvdI5g/edit#gid=341089357"
    google_sheet = google_sheet.replace("edit#gid","export?format=csv&gid")
    df_google = pd.read_csv(google_sheet)
    print(df_google.iloc[0:3])
    print(df_google.columns)
    print(df_google.dtypes)
    print(df_google.describe(include =[np.number]))
    unique_categories = df_google[['Survived','Pclass','Sex','SibSp','Embarked']]
    unique_categories = [unique_categories[i].unique().tolist() for i in unique_categories.columns]
    print(unique_categories)
    return df_google, unique_categories


url = env.get_db_url('titanic_db')

def wrangle_titanic():
    df = pd.read_sql("""

SELECT *
FROM passengers
"""
,url)
    return df


def wrangle_iris2():
    df = pd.read_sql("""

SELECT *
FROM measurements m
JOIN species s on s.species_id = m.species_id;
"""
,url)

In [3]:
df_iris = acquire.wrangle_iris()

In [4]:
df_iris.head(1)

,measurement_id,sepal_length,sepal_width,petal_length,petal_width,species_id,species_id,species_name
0,1,5.1,3.5,1.4,0.2,1,1,setosa


In [6]:
df_iris.drop(['species_id','measurement_id'],axis=1,inplace=True)

In [7]:
df_iris.head(2)

,sepal_length,sepal_width,petal_length,petal_width,species_name
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
